### Steps for execution:
- 1) Import the Rubric which has all the matching EOM fee amounts 
- 2) Import the EOM View dataframe, this is the base of our output
- 3) Needed column from EOM: Processor, Card Type, Merchant Group, Attempted Captured Charges, Processed, Chargebacks, Alerts.
- 4) Columns to calc: Disc Due, Auth Due, CB Due, Visa Alert Due
    - Disc Due --> **Processed** x **Discount Fee**
    - Auth Due --> **Attempted Captured Charges** x **Attemt Fees**
    - CB Due --> **Chargebacks** x **35**
    - Visa Alert Due --> **Alerts** x **Visa Alert**
    - EOM --> **SUM OF ALL**


### Step 1) Import the Rubric Dataframe which we will convert to a dictionary

In [1]:
import os
import pandas as pd 
import numpy as np

# Get the current directory (where the notebook is saved)
current_directory = os.getcwd()

files = os.listdir(current_directory)

# Get the EOM df
for RUBRIC_df in files:
    if "EOM" in RUBRIC_df and "Rubric" in RUBRIC_df:
        print(RUBRIC_df)
        break

# Import the EOM_df 
import pandas as pd
import os

def import_file(file_name):
    """
    Import a specific file regardless of whether it's CSV or Excel
    
    Parameters:
    file_name (str): Name of the file to import
    
    Returns:
    pandas.DataFrame: The imported data
    """
    # Get the full path
    file_path = os.path.join(os.getcwd(), file_name)
    
    # Check if file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File {file_name} not found in the current directory")
    
    # Get the file extension (lowercase)
    _, file_extension = os.path.splitext(file_name)
    file_extension = file_extension.lower()
    
    # Import based on file extension
    if file_extension == '.csv':
        return pd.read_csv(file_path)
    elif file_extension in ['.xlsx', '.xls']:
        return pd.read_excel(file_path)
    else:
        raise ValueError(f"Unsupported file format: {file_extension}")

# Example usage:
try:
    RUBRIC_df = import_file(RUBRIC_df)
    print(f"Successfully imported {RUBRIC_df}")
except Exception as e:
    print(f"Error importing file: {e}")
RUBRIC_df.columns = RUBRIC_df.iloc[1]
RUBRIC_df = RUBRIC_df[2:]
RUBRIC_df

# restaret the index 
RUBRIC_df.reset_index(drop=True, inplace=True)

# make sure that the Processor column is a string 
RUBRIC_df['Processor'] = RUBRIC_df['Processor'].astype(str)

RUBRIC_df

GF_EOM-Rubric.xlsx
Successfully imported           EOM Rubric     Unnamed: 1    Unnamed: 2 Unnamed: 3      Unnamed: 4
0                NaN            NaN           NaN        NaN             NaN
1          Processor  Discount Fees  Attempt Fees     CB Fee  Visa Alert Due
2            PAYSAFE              0           0.6         35              20
3             PAYARC              0             0          0               0
4               APPS              0             0          0               0
5            NETEVIA              0           0.5          0               0
6           PRIORITY            NaN           NaN        NaN             NaN
7            QUANTUM           0.07           0.6         35              15
8           SignaPay              0          0.65         35              20
9           Maverick              0           0.5         35              20
10      PAYSAFE_BBVA              0           0.6         35              20
11       PAYSAFE_PNC              0

1,Processor,Discount Fees,Attempt Fees,CB Fee,Visa Alert Due
0,PAYSAFE,0,0.6,35,20
1,PAYARC,0,0,0,0
2,APPS,0,0,0,0
3,NETEVIA,0,0.5,0,0
4,PRIORITY,NaN,NaN,NaN,NaN
5,QUANTUM,0.07,0.6,35,15
6,SignaPay,0,0.65,35,20
7,Maverick,0,0.5,35,20
8,PAYSAFE_BBVA,0,0.6,35,20
9,PAYSAFE_PNC,0,0.6,35,20


In [2]:
# make sure that the remainig column are in float 
RUBRIC_df.iloc[:, 1:] = RUBRIC_df.iloc[:, 1:].astype(float)
what = RUBRIC_df.loc[5, "Processor"]

# We are goiung to convert the RUBRIC_df into a dictionary
RUBRIC_dict = RUBRIC_df.set_index("Processor").T.to_dict()

# Set all values into float 
RUBRIC_dict = {
    key.lower(): {inner_key: float(inner_value) for inner_key, inner_value in value.items()}
    for key, value in RUBRIC_dict.items()
}

RUBRIC_dict = {key.lower(): value for key, value in RUBRIC_dict.items()}
RUBRIC_dict["merchant industries"] = {
    "Discount Fees": 0.0,
    "Attempt Fees": 0.0,
    "CB Fee": 35.0,
    "Visa Alert Due": 0.0
}

RUBRIC_dict

C:\Users\mmsou\AppData\Local\Temp\ipykernel_4400\4245926676.py:6: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  RUBRIC_dict = RUBRIC_df.set_index("Processor").T.to_dict()


{'paysafe': {'Discount Fees': 0.0,
  'Attempt Fees': 0.6,
  'CB Fee': 35.0,
  'Visa Alert Due': 20.0},
 'payarc': {'Discount Fees': 0.0,
  'Attempt Fees': 0.0,
  'CB Fee': 0.0,
  'Visa Alert Due': 0.0},
 'apps': {'Discount Fees': 0.0,
  'Attempt Fees': 0.0,
  'CB Fee': 0.0,
  'Visa Alert Due': 0.0},
 'netevia': {'Discount Fees': 0.0,
  'Attempt Fees': 0.5,
  'CB Fee': 0.0,
  'Visa Alert Due': 0.0},
 'priority': {'Discount Fees': nan,
  'Attempt Fees': nan,
  'CB Fee': nan,
  'Visa Alert Due': nan},
 'quantum': {'Discount Fees': 0.07,
  'Attempt Fees': 0.6,
  'CB Fee': 35.0,
  'Visa Alert Due': 15.0},
 'signapay': {'Discount Fees': 0.0,
  'Attempt Fees': 0.65,
  'CB Fee': 35.0,
  'Visa Alert Due': 20.0},
 'maverick': {'Discount Fees': 0.0,
  'Attempt Fees': 0.5,
  'CB Fee': 35.0,
  'Visa Alert Due': 20.0},
 'paysafe_bbva': {'Discount Fees': 0.0,
  'Attempt Fees': 0.6,
  'CB Fee': 35.0,
  'Visa Alert Due': 20.0},
 'paysafe_pnc': {'Discount Fees': 0.0,
  'Attempt Fees': 0.6,
  'CB Fee': 3

### Step 2) Import the EOM dataframe

In [3]:
# Get the EOM df
for EOM_df in files:
    if "EOM" in EOM_df and "View" in EOM_df:
        print(EOM_df)
        break

In [4]:
current_directory = os.getcwd()
current_directory

'c:\\Users\\mmsou\\Documents\\mokum.ai\\Goldfinger'

In [5]:
import pandas as pd
import os

# Get the current directory (where the notebook is saved)
current_directory = os.getcwd()

# List all files in the current directory
files = os.listdir(current_directory)

# Import the EOM_df
def import_file(file_name):
    if file_name.endswith(".csv"):
        return pd.read_csv(file_name)
    elif file_name.endswith((".xls", ".xlsx")):
        return pd.read_excel(file_name)
    else:
        raise ValueError("Unsupported file format")

# Example usage: Look for a file named "EOM-View_EXPORT.csv" or similar
for file in files:
    if "EOM" in file and file.endswith((".csv", ".xls", ".xlsx")) and "Fcast" in file:
        EOM_df = import_file(file)
        break
else:
    raise FileNotFoundError("No EOM file found in the current directory")

# Convert "Processor", "Card Type", and "Merchant Group" into string columns
EOM_df["Processor"] = EOM_df["Processor"].astype(str)
EOM_df["Card Type"] = EOM_df["Card Type"].astype(str)
EOM_df["Merchant Group"] = EOM_df["Merchant Group"].astype(str)


- Disc Due --> **Processed** x **Discount Fee**
- Auth Due --> **Attempted Captured Charges** x **Attempt Fees**
- CB Due --> **Chargebacks** x **35**
- Visa Alert Due --> **Alerts** x **Visa Alert**
- EOM --> **SUM OF ALL**

As we through the df we append the index and values for each dict representing each new colummn, in the end we will map the original df with these to fill them out

In [6]:
EOM_df = EOM_df[EOM_df['Merchant Group'] != "Sale Shield"]
EOM_df = EOM_df[EOM_df['Merchant Group'] != "SalesShield"]
EOM_df = EOM_df[EOM_df["Processor"] != "EMS"]

In [7]:
EOM_df.columns

# Columns to keep Processor, Card Type, Merchant Group, Attempted Captured Charges, Processed, Chargebacks, Alerts, Disc Due, Auth Due, CB Due, Visa Alert Due
EOM_df = EOM_df[["Processor", "Card Type", "Merchant Group", "Attempted Captured Charges", "Processed", "Chargebacks", "Alerts"]]
EOM_df["Disc Due"] = np.nan
EOM_df["Auth Due"] = np.nan
EOM_df["CB Due"] = np.nan
EOM_df["Visa Alert Due"] = np.nan
EOM_df["Total EOM"] = np.nan

# Drop all rows that have FlexFactor and Stripe in the processor column
EOM_df = EOM_df[~EOM_df["Processor"].str.contains("FlexFactor|Stripe", na=False)]

# Remove trailing spaces from EOM_df column names
def clean_column_names(df):
    df.columns = (
        df.columns.str.strip()         # Remove leading/trailing spaces
        .str.lower()                   # Convert to lowercase
        .str.replace(r'\W+', '_', regex=True)  # Replace non-word characters with '_'
        .str.replace(r'_+', '_', regex=True)   # Remove multiple consecutive '_'
        .str.rstrip('_')                # Remove trailing '_'
    )
    return df

# Clean Porcessed column such that we remove special characters and convert to float
EOM_df["Processed"] = EOM_df["Processed"].astype(str).str.replace(r'[^0-9.]', '', regex=True).astype(float)

# Replace empty strings and non-numeric values with NaN
EOM_df["Attempted Captured Charges"] = (
    EOM_df["Attempted Captured Charges"]
    .astype(str)  # Ensure the column is treated as strings
    .str.replace(r'[^0-9.]', '', regex=True)  # Remove non-numeric characters
    .replace('', 0)  # Replace empty strings with NaN
)

# Convert the column to float, coercing any remaining invalid values to NaN
EOM_df["Attempted Captured Charges"] = pd.to_numeric(EOM_df["Attempted Captured Charges"], errors='coerce')

# Convert charbacks column into integer 
# Clean and convert the column (replace "Chargebacks" with the desired column name)
EOM_df["Chargebacks"] = (
    EOM_df["Chargebacks"]
    .astype(str)  # Ensure the column is treated as strings
    .str.replace(r'[^0-9.]', '', regex=True)  # Remove non-numeric characters
    .replace('', 0)  # Replace empty strings with NaN
    .astype(float)  # Convert to float
)
EOM_df["Chargebacks"] = EOM_df["Chargebacks"].astype(str).str.replace(r'[^0-9.]', '', regex=True).astype(float)

EOM_df = clean_column_names(EOM_df)


In [8]:
# All teh rpocessors from the rubric 
processors = RUBRIC_df["Processor"].unique()
processors

processors_in_EOM = EOM_df["processor"].unique()

# now we need to check if all the processors in the rubric are in the EOM df
print("Processors in Rubric:")
print(processors)
print("\nProcessors in EOM:")
print(processors_in_EOM)

# Check for missing processors
missing_processors = []
for processor in processors:
    if processor.lower() not in [p.lower() for p in processors_in_EOM]:
        missing_processors.append(processor)

if missing_processors:
    print(f"\n⚠️ Warning: The following processors from the rubric are not found in EOM data:")
    for proc in missing_processors:
        print(f"  - {proc}")
else:
    print("\n✅ All processors from the rubric are present in the EOM data")

# Check for processors in EOM that are not in the rubric
extra_processors = []
for processor in processors_in_EOM:
    if processor.lower() not in [p.lower() for p in processors]:
        extra_processors.append(processor)

if extra_processors:
    print(f"\n⚠️ Warning: The following processors in EOM data are not in the rubric:")
    for proc in extra_processors:
        print(f"  - {proc}")
else:
    print("\n✅ All processors in EOM data are covered by the rubric")

# Now let's calculate the fees for each row
print("\n🔄 Calculating fees for each row...")

# Initialize counters for tracking
processed_rows = 0
error_rows = 0
errors = []

for index, row in EOM_df.iterrows():
    try:
        processor = row['processor'].lower()
        
        # Get the fee structure for this processor
        if processor in RUBRIC_dict:
            fees = RUBRIC_dict[processor]
        else:
            # Try to find a partial match
            matching_processor = None
            for rubric_proc in RUBRIC_dict.keys():
                if processor in rubric_proc or rubric_proc in processor:
                    matching_processor = rubric_proc
                    break
            
            if matching_processor:
                fees = RUBRIC_dict[matching_processor]
                print(f"  📝 Using fees from '{matching_processor}' for processor '{processor}'")
            else:
                # Use default fees for unknown processors
                fees = {
                    "Discount Fees": 0.0,
                    "Attempt Fees": 0.0,
                    "CB Fee": 35.0,
                    "Visa Alert Due": 0.0
                }
                print(f"  ⚠️ No fee structure found for processor '{processor}', using defaults")
        
        # Calculate fees
        processed_amount = row['processed'] if pd.notna(row['processed']) else 0
        attempted_charges = row['attempted_captured_charges'] if pd.notna(row['attempted_captured_charges']) else 0
        chargebacks = row['chargebacks'] if pd.notna(row['chargebacks']) else 0
        alerts = row['alerts'] if pd.notna(row['alerts']) else 0
        
        # Calculate each fee type
        disc_due = processed_amount * fees["Discount Fees"]
        auth_due = attempted_charges * fees["Attempt Fees"]
        cb_due = chargebacks * fees["CB Fee"]
        visa_alert_due = alerts * fees["Visa Alert Due"]
        total_eom = disc_due + auth_due + cb_due + visa_alert_due
        
        # Update the row
        EOM_df.at[index, 'disc_due'] = disc_due
        EOM_df.at[index, 'auth_due'] = auth_due
        EOM_df.at[index, 'cb_due'] = cb_due
        EOM_df.at[index, 'visa_alert_due'] = visa_alert_due
        EOM_df.at[index, 'total_eom'] = total_eom
        
        processed_rows += 1
        
    except Exception as e:
        error_rows += 1
        errors.append(f"Row {index}: {str(e)}")
        print(f"  ❌ Error processing row {index}: {str(e)}")

print(f"\n✅ Processing complete:")
print(f"  - Successfully processed: {processed_rows} rows")
print(f"  - Errors: {error_rows} rows")

if errors:
    print("\n❌ Errors encountered:")
    for error in errors[:5]:  # Show first 5 errors
        print(f"  {error}")
    if len(errors) > 5:
        print(f"  ... and {len(errors) - 5} more errors")

# Display summary statistics
print("\n📊 Summary Statistics:")
print(f"Total EOM fees: ${EOM_df['total_eom'].sum():,.2f}")
print(f"Average EOM per row: ${EOM_df['total_eom'].mean():,.2f}")
print(f"Max EOM per row: ${EOM_df['total_eom'].max():,.2f}")
print(f"Min EOM per row: ${EOM_df['total_eom'].min():,.2f}")

# Show breakdown by fee type
print("\n💰 Fee Breakdown:")
print(f"Discount fees: ${EOM_df['disc_due'].sum():,.2f}")
print(f"Authorization fees: ${EOM_df['auth_due'].sum():,.2f}")
print(f"Chargeback fees: ${EOM_df['cb_due'].sum():,.2f}")
print(f"Visa Alert fees: ${EOM_df['visa_alert_due'].sum():,.2f}")

# Display the final dataframe
print("\n📋 Final EOM DataFrame:")
EOM_df.head(10)

Processors in Rubric:
['PAYSAFE' 'PAYARC' 'APPS' 'NETEVIA' 'PRIORITY' 'QUANTUM' 'SignaPay'
 'Maverick' 'PAYSAFE_BBVA' 'PAYSAFE_PNC' 'APPS_SYNOVOUS' 'APPS_SYNOVUS'
 'NETEVIA_ESQUIRE' 'PAYARC_EVOLVE' 'PRIORITY_SYNOVUS' 'QUANTUM_CBSL'
 'QUANTUM_FRESNO' 'SIGNAPAY' 'PayArc' 'Luqra']

Processors in EOM:
['APPS' 'Luqra' 'NETEVIA' 'PAYARC' 'PAYSAFE' 'PRIORITY' 'PayArc' 'QUANTUM'
 'SIGNAPAY']

⚠️ Warning: The following processors from the rubric are not found in EOM data:
  - Maverick
  - PAYSAFE_BBVA
  - PAYSAFE_PNC
  - APPS_SYNOVOUS
  - APPS_SYNOVUS
  - NETEVIA_ESQUIRE
  - PAYARC_EVOLVE
  - PRIORITY_SYNOVUS
  - QUANTUM_CBSL
  - QUANTUM_FRESNO

✅ All processors in EOM data are covered by the rubric

🔄 Calculating fees for each row...

✅ Processing complete:
  - Successfully processed: 62 rows
  - Errors: 0 rows

📊 Summary Statistics:
Total EOM fees: $53,251.83
Average EOM per row: $1,024.07
Max EOM per row: $4,795.03
Min EOM per row: $0.00

💰 Fee Breakdown:
Discount fees: $9,924.88
Authorizati

,processor,card_type,merchant_group,attempted_captured_charges,processed,chargebacks,alerts,disc_due,auth_due,cb_due,visa_alert_due,total_eom
0,APPS,Mastercard,SpecifiConLLC,883,13227.01,6.0,83,0.000,0.0,0.0,0.0,0.000
1,APPS,Visa,SpecifiConLLC,2038,11804.73,0.0,54,0.000,0.0,0.0,0.0,0.000
2,Luqra,Mastercard,nan,598,6121.55,0.0,0,244.862,358.8,0.0,0.0,603.662
3,Luqra,Visa,nan,1165,3547.20,0.0,0,141.888,699.0,0.0,0.0,840.888
4,NETEVIA,Mastercard,BrightAdvantageLLC,1247,18393.87,24.0,92,0.000,623.5,0.0,0.0,623.500
5,NETEVIA,Mastercard,DiamondSphereLLC,1313,23559.78,10.0,115,0.000,656.5,0.0,0.0,656.500
6,NETEVIA,Mastercard,PrimeSmartSolutionsLLC,907,19725.56,4.0,67,0.000,453.5,0.0,0.0,453.500
7,NETEVIA,Mastercard,SpecifiConLLC,1072,16005.01,5.0,88,0.000,536.0,0.0,0.0,536.000
8,NETEVIA,Mastercard,UniquePlusLLC,848,14944.61,6.0,67,0.000,424.0,0.0,0.0,424.000
9,NETEVIA,Visa,BrightAdvantageLLC,2621,11885.49,0.0,26,0.000,1310.5,0.0,0.0,1310.500


In [9]:
# If nan in the processor column, then drop the row
EOM_df = EOM_df.dropna(subset=["processor"])
# Reset the index
EOM_df.reset_index(drop=True, inplace=True)
EOM_df 

,processor,card_type,merchant_group,attempted_captured_charges,processed,chargebacks,alerts,disc_due,auth_due,cb_due,visa_alert_due,total_eom
0,APPS,Mastercard,SpecifiConLLC,883,13227.01,6.0,83,0.000,0.00,0.0,0.0,0.000
1,APPS,Visa,SpecifiConLLC,2038,11804.73,0.0,54,0.000,0.00,0.0,0.0,0.000
2,Luqra,Mastercard,nan,598,6121.55,0.0,0,244.862,358.80,0.0,0.0,603.662
3,Luqra,Visa,nan,1165,3547.20,0.0,0,141.888,699.00,0.0,0.0,840.888
4,NETEVIA,Mastercard,BrightAdvantageLLC,1247,18393.87,24.0,92,0.000,623.50,0.0,0.0,623.500
...,...,...,...,...,...,...,...,...,...,...,...,...
57,SIGNAPAY,Mastercard,UniquePlusLLC,450,7630.68,6.0,54,0.000,292.50,210.0,1080.0,1582.500
58,SIGNAPAY,Visa,BrightAdvantageLLC,1292,8313.71,4.0,28,0.000,839.80,140.0,560.0,1539.800
59,SIGNAPAY,Visa,DiamondSphereLLC,1558,9671.87,2.0,29,0.000,1012.70,70.0,580.0,1662.700
60,SIGNAPAY,Visa,SpecifiConLLC,101,637.97,6.0,1,0.000,65.65,210.0,20.0,295.650


#### Filling in the columns 

In [10]:
# Now we fill in the columns above

# These dicts will in the end be mapped over the original df, they have as keys the index and as values the due amounts
disc_due_dict = {}
auth_due_dict = {}
cb_due_dict = {}
visa_alert_due_dict = {}


# Debug 1: Check if all processors in df are in the dict
count_debug_1 = 0
# Debug 2: Check if totals match rod output
count_debug_2 = 0

# Debug 3: Check if total match len of df 
count_debug3 = 0 

# Debug 4: Check if total match len of df
count_debug4 = 0

for row in EOM_df.itertuples():
    P = row.processor.lower()
    amount_processed = row.processed
    attempted_captured_charges = row.attempted_captured_charges
    attempted_captured_charges = float(attempted_captured_charges)
    chargebacks = row.chargebacks
    alerts = row.alerts
    card_used = row.card_type.lower()
    

    # start the iteration
    if P in RUBRIC_dict:
        count_debug_1 += 1

        # Column Disc Due --> Discount on the processed column
        discount_due = RUBRIC_dict[P]["Discount Fees"] * amount_processed
        count_debug_2 += discount_due
        disc_due_dict[row.Index] = discount_due

        # Column Auth Due --> Authorization on the processed column
        authorization_due = RUBRIC_dict[P]["Attempt Fees"] * attempted_captured_charges
        count_debug3 += 1
        auth_due_dict[row.Index] = authorization_due

        # Column CB Due --> Chargeback on the processed column
        chargeback_due = RUBRIC_dict[P]["CB Fee"] * chargebacks
        cb_due_dict[row.Index] = chargeback_due
        count_debug4 += 1
        cb_due_dict[row.Index] = chargeback_due

        # Column Visa Alert Due --> Visa Alert on the processed column
        if "visa" in card_used:
            
            visa_alert_due = RUBRIC_dict[P]["Visa Alert Due"] * alerts
            visa_alert_due_dict[row.Index] = visa_alert_due
        else:
            visa_alert_due = 0.0
            visa_alert_due_dict[row.Index] = visa_alert_due
    else:
        print(P)
    

print(count_debug_1, EOM_df.shape[0], count_debug_2)
print(f"Debug 1, passed? {count_debug_1==EOM_df.shape[0]} ")

# Debug 2: Check if the len of the dict is equal to the number of rows in the df
print(f"Debug 2, passed? {len(disc_due_dict)==EOM_df.shape[0]} ")

# Debug 3: Check if the total of the dict is equal to the number of rows in the df
print(f"Debug 3, passed? {len(auth_due_dict)==EOM_df.shape[0]} ")




62 62 nan
Debug 1, passed? True 
Debug 2, passed? True 
Debug 3, passed? True 


In [11]:
EOM_df[EOM_df["merchant_group"] == "nan"]

,processor,card_type,merchant_group,attempted_captured_charges,processed,chargebacks,alerts,disc_due,auth_due,cb_due,visa_alert_due,total_eom
2,Luqra,Mastercard,nan,598,6121.55,0.0,0,244.862,358.8,0.0,0.0,603.662
3,Luqra,Visa,nan,1165,3547.20,0.0,0,141.888,699.0,0.0,0.0,840.888
44,PayArc,Mastercard,nan,630,2172.37,0.0,0,0.000,0.0,0.0,0.0,0.000
45,PayArc,Visa,nan,1153,7056.19,0.0,0,0.000,0.0,0.0,0.0,0.000


In [12]:
EOM_df.tail(60)

,processor,card_type,merchant_group,attempted_captured_charges,processed,chargebacks,alerts,disc_due,auth_due,cb_due,visa_alert_due,total_eom
2,Luqra,Mastercard,nan,598,6121.55,0.0,0,244.8620,358.80,0.0,0.0,603.6620
3,Luqra,Visa,nan,1165,3547.20,0.0,0,141.8880,699.00,0.0,0.0,840.8880
4,NETEVIA,Mastercard,BrightAdvantageLLC,1247,18393.87,24.0,92,0.0000,623.50,0.0,0.0,623.5000
5,NETEVIA,Mastercard,DiamondSphereLLC,1313,23559.78,10.0,115,0.0000,656.50,0.0,0.0,656.5000
6,NETEVIA,Mastercard,PrimeSmartSolutionsLLC,907,19725.56,4.0,67,0.0000,453.50,0.0,0.0,453.5000
7,NETEVIA,Mastercard,SpecifiConLLC,1072,16005.01,5.0,88,0.0000,536.00,0.0,0.0,536.0000
8,NETEVIA,Mastercard,UniquePlusLLC,848,14944.61,6.0,67,0.0000,424.00,0.0,0.0,424.0000
9,NETEVIA,Visa,BrightAdvantageLLC,2621,11885.49,0.0,26,0.0000,1310.50,0.0,0.0,1310.5000
10,NETEVIA,Visa,DiamondSphereLLC,2589,13901.60,3.0,53,0.0000,1294.50,0.0,0.0,1294.5000
11,NETEVIA,Visa,PrimeSmartSolutionsLLC,1654,8763.69,1.0,28,0.0000,827.00,0.0,0.0,827.0000


### Processor Dataframe 

In [13]:
EOM_df["total_eom"].sum()

53251.8321

In [14]:
# Group EOM df by processor
selected_c = ["processor", "total_eom"]
EOM_processor = EOM_df[selected_c].groupby("processor").sum().reset_index()

# If we have a nan in the processor column, we remove that row 
EOM_processor = EOM_processor.dropna(subset=["processor"])
EOM_processor = EOM_processor[EOM_processor["processor"].str.lower() != "nan"]
total_row_processor = pd.DataFrame({
    'processor': ['Total'],
    'total_eom': [EOM_processor['total_eom'].sum()]
})

# Append the total row to the original DataFrame
EOM_processor = pd.concat([EOM_processor, total_row_processor], ignore_index=True)
EOM_processor["processor"] = EOM_processor["processor"].str.upper()
EOM_processor
# Clean up


,processor,total_eom
0,APPS,0.0000
1,LUQRA,1444.5500
2,NETEVIA,8019.0000
3,PAYARC,0.0000
4,PAYSAFE,4020.4000
5,PRIORITY,0.0000
6,PAYARC,0.0000
7,QUANTUM,28366.9321
8,SIGNAPAY,11400.9500
9,TOTAL,53251.8321


In [15]:
# First, let's clean the processor column in the main dataframe
EOM_df["processor"] = EOM_df["processor"].astype(str).str.strip().str.lower()

# Now group by processor (which is now cleaned)
selected_c = ["processor", "total_eom"]
EOM_processor = EOM_df[selected_c].groupby("processor").sum().reset_index()

# If we have a nan in the processor column, we remove that row 
EOM_processor = EOM_processor.dropna(subset=["processor"])
EOM_processor = EOM_processor[EOM_processor["processor"].str.lower() != "nan"]

# Add total row
total_row_processor = pd.DataFrame({
    'processor': ['total'],
    'total_eom': [EOM_processor['total_eom'].sum()]
})

# Append the total row to the original DataFrame
EOM_processor = pd.concat([EOM_processor, total_row_processor], ignore_index=True)

# Convert to proper case (title case for processors, uppercase for TOTAL)
EOM_processor["processor"] = EOM_processor["processor"].apply(
    lambda x: x.title() if x.lower() == 'total' else x.title()
)

EOM_processor

,processor,total_eom
0,Apps,0.0000
1,Luqra,1444.5500
2,Netevia,8019.0000
3,Payarc,0.0000
4,Paysafe,4020.4000
5,Priority,0.0000
6,Quantum,28366.9321
7,Signapay,11400.9500
8,Total,53251.8321


#### Corp dataframe

In [16]:
selected_c = ["merchant_group", "total_eom"]
EOM_merchant_group = EOM_df[selected_c].groupby("merchant_group").sum().reset_index()

# order merchant group in alphabetical order
EOM_merchant_group = EOM_merchant_group.sort_values("merchant_group").reset_index(drop=True)

# if nan in merchant group delete the row 
"""
EOM_merchant_group = EOM_merchant_group[
    EOM_merchant_group["merchant_group"].notna() &  # Remove actual NaN values
    (EOM_merchant_group["merchant_group"].str.lower() != "nan")  # Remove string "nan"
]
"""

total_row = pd.DataFrame({
    'merchant_group': ['Total'],
    'total_eom': [EOM_merchant_group['total_eom'].sum()]
})
EOM_merchant_group = pd.concat([EOM_merchant_group, total_row], ignore_index=True)
EOM_merchant_group

,merchant_group,total_eom
0,BrightAdvantageLLC,13193.2167
1,DiamondSphereLLC,14037.8144
2,PrimeSmartSolutionsLLC,2732.5000
3,SpecifiConLLC,9636.6158
4,UniquePlusLLC,12207.1352
5,nan,1444.5500
6,Total,53251.8321


#### Convert to message 


In [17]:
message_p3 = ""
message_p3 += f"Total EOM fees by Processor: \n"

# Format processor-level fees
for row in EOM_processor.itertuples():
    if not pd.isna(row.processor):  # Exclude rows with missing processor values
        message_p3 += f"{row.processor}: ${row.total_eom:,.2f}\n"

message_p3 += "----------------------------------\n"
message_p3 += "Total EOM fees by Corp: \n"

# Format corp-level fees
for row in EOM_merchant_group.itertuples():
    if not pd.isna(row.merchant_group):  # Exclude rows with missing merchant_group values
        message_p3 += f"{row.merchant_group}: ${row.total_eom:,.2f}\n"

print(message_p3)

Total EOM fees by Processor: 
Apps: $0.00
Luqra: $1,444.55
Netevia: $8,019.00
Payarc: $0.00
Paysafe: $4,020.40
Priority: $0.00
Quantum: $28,366.93
Signapay: $11,400.95
Total: $53,251.83
----------------------------------
Total EOM fees by Corp: 
BrightAdvantageLLC: $13,193.22
DiamondSphereLLC: $14,037.81
PrimeSmartSolutionsLLC: $2,732.50
SpecifiConLLC: $9,636.62
UniquePlusLLC: $12,207.14
nan: $1,444.55
Total: $53,251.83



In [18]:
# Create a excel file that has three sheets, sheet 1 is EOM_df, sheet 2 is EOM_processor, sheet 3 is EOM_merchant_group
# Create an Excel file with three sheets
output_file = "EOM_Report.xlsx"
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    EOM_df.to_excel(writer, sheet_name="EOM_df", index=False)
    EOM_processor.to_excel(writer, sheet_name="EOM_processor", index=False)
    EOM_merchant_group.to_excel(writer, sheet_name="EOM_merchant_group", index=False)

print(f"✅ Excel file '{output_file}' created successfully!")


✅ Excel file 'EOM_Report.xlsx' created successfully!


### Export 

In [19]:
# Create a excel file that has three sheets, sheet 1 is EOM_df, sheet 2 is EOM_processor, sheet 3 is EOM_merchant_group
# Create an Excel file with three sheets
output_file = "EOM_Report.xlsx"
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    EOM_df.to_excel(writer, sheet_name="EOM_df", index=False)
    EOM_processor.to_excel(writer, sheet_name="EOM_processor", index=False)
    EOM_merchant_group.to_excel(writer, sheet_name="EOM_merchant_group", index=False)

print(f"✅ Excel file '{output_file}' created successfully!")


✅ Excel file 'EOM_Report.xlsx' created successfully!
